## Parte 1 :   Importando Bibliotecas que serão usadas

In [96]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv

import re
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
from nltk.tokenize import word_tokenize
from nltk import tokenize 
from nltk.stem import RSLPStemmer
from nltk.stem.porter import *
import heapq
import time
%matplotlib inline


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!



## Lendo arquivo csv
## Eliminando caracteres com tamanho 2 ou menor e deixando todos em caixa baixa




In [97]:
arquivo = pd.read_csv('results.csv')
texto = arquivo['text']

def eliminar_carac(text):
  return re.sub(r'(\W+|\d+|\b\w{0,2}\b)', ' ', text.lower())
texto = texto.apply(eliminar_carac)

texto = texto.apply(tokenize.word_tokenize, language='portuguese')
palavras = []


texto.head()

0    [juíza, federal, ivani, silva, luz, brasília, ...
1    [menos, horas, depois, juíza, federal, ivani, ...
2    [quando, determinou, que, março, uma, estratég...
3    [quando, era, pequeno, luisa, gabriel, preocup...
4    [desde, manhã, última, segunda, feira, sucesso...
Name: text, dtype: object

## Eliminando as stopwords : Usando a biblioteca nltk
## Algoritmo Build Index : Usada para uma indexação simples


In [0]:
stopwords = nltk.corpus.stopwords.words('portuguese')

def eliminar_stopwords(text):
  palavras = []
  for palavra in text:
    if(palavra not in stopwords):
      palavras.append(palavra)
  return palavras 

def build_index(arquivo):
  numero_da_lista = 0
  dic_index = {}
  
  for documento in arquivo:
    
    numero_da_lista += 1
    doc = eliminar_stopwords(documento)
    
    for palavra in doc:
      ocorrencia = doc.count(palavra)
      if palavra not in dic_index:
        dic_index[palavra] = []
      par = (numero_da_lista, ocorrencia)
      if par not in dic_index[palavra]:
         dic_index[palavra].append(par)
  return dic_index

index = build_index(texto)
tabela = {'palavra' : list(index.keys()),
       'doc|ocorrencia' : list(index.values()) }
df = pd.DataFrame(tabela)
df.head()
# df.to_csv('index.csv')


# Parte 2 : Implementação do processamento de consulta documento-por-vez e termo-por-vez



## Documento Por Vez : Calcula escores completos para documentos processando todas as listas de termos, um documento por vez

In [100]:
def document_por_vez(Q, I, k):
    L = []
    R = []
    for word in Q.split(" "):
        if word in I.keys():
            L.append(I[word])
    for document in range(1, len(texto)+1):
        sd = 0
        for l in L:
            for i in l:
                if (i[0] == document):
                    sd += i[1]
                    break
        if (sd != 0):
          heapq.heappush(R, (sd, document))
    return heapq.nlargest(k, R)
document_por_vez("onu", index, 10)

[(2, 101),
 (2, 64),
 (2, 29),
 (1, 249),
 (1, 185),
 (1, 97),
 (1, 70),
 (1, 42),
 (1, 8),
 (1, 3)]

## Termo Por Vez: Acumula escores para documentos processando listas de termos, um termo por vez

In [101]:
def termo_por_vez(Q, I, k):
    A = {}
    L = []
    R = []
    for word in Q.split(" "):
        if word in I.keys():
            L.append(I[word])
    for l in L:
        for post in l:
            d = post[0]
            freq = post[1]
            if (d in A.keys()):
                A[d] += freq
            else:
                A[d] = freq
    for (d, ad) in A.items():
        sd = ad
        heapq.heappush(R, (sd, d))
    return heapq.nlargest(k, R)

termo_por_vez("onu", index, 10)

[(2, 101),
 (2, 64),
 (2, 29),
 (1, 249),
 (1, 185),
 (1, 97),
 (1, 70),
 (1, 42),
 (1, 8),
 (1, 3)]

##Definindo consultas de um termo 

In [0]:
queries = ["política", "presidente", "bolsonaro", "maduro", "onu"]

## Executa 5 consultas em cada algoritmo retornando os top-10 

In [0]:
  # Documento
  result_doc = []
  time_doc = []
  # Termo
  result_term = []
  time_term = []
  k = 10
  for query in queries:
    # Documento por vez
    inicio_doc = time.time()
    result_doc.append(document_por_vez(query, index, k))
    fim_doc = time.time()
    time_doc.append(fim_doc - inicio_doc)
    # Termo Por Vez
    inicio_term = time.time()
    result_term.append(termo_por_vez(query, index, k))
    fim_term = time.time()
    time_term.append(fim_term - inicio_term)
  # criação da tabela
  tabela = pd.DataFrame()
  tabela['query'] =  queries
  tabela['documento_por_vez'] = result_doc
  tabela['termo_por_vez'] = result_term
  tabela['comparando'] = tabela.documento_por_vez == tabela.termo_por_vez

## Resultado Top-10 documentos

In [0]:
tabela
# tabela.to_csv('queries.csv')

## Comparando os tempos médios de execução

In [91]:
tabela_comparar = {
    'tempo médio documento': time_doc,
    'tempo médio termo' : time_term
}

tabela_df = pd.DataFrame(tabela_comparar)
tabela_df

,tempo médio documento,tempo médio termo
0,0.001333,0.000117
1,0.001993,0.000122
2,0.001454,0.000131
3,0.000303,0.000021
4,0.000234,0.000013


##Chegando a conclusão que o tempo medio do algoritmo termo-por-vez  é menor que documento-por-vez 

#Parte 3 :  Consulta conjuntiva